In [2]:
#Agregamos las bibliotecas que vayamos a usar
from google.cloud import bigquery
client = bigquery.Client()

In [3]:
#C
dataset_ref = client.dataset("bigquery-public-data.covid19_public_forecasts")
